# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

print(geoapify_key)

91bbe9337d5241559f0714fe7db40d0c


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,21.56,76,100,6.07,ZA,1675792541
1,1,jamestown,42.0970,-79.2353,8.90,52,75,6.17,US,1675792422
2,2,sao miguel do araguaia,-13.2750,-50.1628,31.54,54,83,0.66,BR,1675792541
3,3,nikolskoye,59.7035,30.7861,-2.06,91,90,4.03,RU,1675792541
4,4,constitucion,-35.3333,-72.4167,21.84,52,4,2.11,CL,1675792542


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600 ,
    size= 'Humidity',
    color= 'City'
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# Between 15.5 and 24 degrees celsius
# Between 40% and 60% humidity 
filtered_city_data = city_data_df[(city_data_df['Max Temp'] >= 15.5) & (city_data_df['Max Temp'] <= 24) & (city_data_df['Humidity'] >= 40) & (city_data_df['Humidity'] <= 60)]

# Drop any rows with null values
filtered_city_data.dropna(inplace=True)

# Display sample data
filtered_city_data

c:\Users\Patrick\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,constitucion,-35.3333,-72.4167,21.84,52,4,2.11,CL,1675792542
24,24,madison,34.7334,-86.5667,19.36,44,0,6.69,US,1675792418
99,99,huntsville,34.7304,-86.5861,19.61,45,0,6.69,US,1675792492
132,132,rio gallegos,-51.6226,-69.2181,20.03,42,40,0.93,AR,1675792611
198,198,puerto del rosario,28.5004,-13.8627,17.86,48,20,6.69,ES,1675792644
263,263,mecca,21.4267,39.8261,19.89,51,100,6.13,SA,1675792672
308,308,tura,25.5198,90.2201,15.88,46,1,1.79,IN,1675792690
312,312,praya,14.9215,-23.5087,23.30,49,40,12.35,CV,1675792691
392,392,marzuq,14.4000,46.4667,18.11,53,34,2.90,YE,1675792716
456,456,avera,33.1940,-82.5271,19.75,42,73,2.99,US,1675792735


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  filtered_city_data[['City' , 'Country', 'Lat' , 'Lng', 'Humidity' ]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df['Lng']

4     -72.4167
24    -86.5667
99    -86.5861
132   -69.2181
198   -13.8627
263    39.8261
308    90.2201
312   -23.5087
392    46.4667
456   -82.5271
561    76.9500
Name: Lng, dtype: float64

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories= "accommodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit": limit,
    "filter": '',
    "bias": '',
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat'][index]
    longitude = hotel_df['Lng'][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"

    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url , params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url , params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
constitucion - nearest hotel: Hostal Alameda
madison - nearest hotel: 106 Jefferson Huntsville, Curio Collection by Hilton
huntsville - nearest hotel: 106 Jefferson Huntsville, Curio Collection by Hilton
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
puerto del rosario - nearest hotel: Hotel Tamasite
mecca - nearest hotel: Rehab Al Marwa Hotel 1
tura - nearest hotel: No hotel found
praya - nearest hotel: Casa Sodadi
marzuq - nearest hotel: No hotel found
avera - nearest hotel: No hotel found
chamrajnagar - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,constitucion,CL,-35.3333,-72.4167,52,Hostal Alameda
24,madison,US,34.7334,-86.5667,44,"106 Jefferson Huntsville, Curio Collection by ..."
99,huntsville,US,34.7304,-86.5861,45,"106 Jefferson Huntsville, Curio Collection by ..."
132,rio gallegos,AR,-51.6226,-69.2181,42,Amérian Río Gallegos Apart & Suites
198,puerto del rosario,ES,28.5004,-13.8627,48,Hotel Tamasite
263,mecca,SA,21.4267,39.8261,51,Rehab Al Marwa Hotel 1
308,tura,IN,25.5198,90.2201,46,No hotel found
312,praya,CV,14.9215,-23.5087,49,Casa Sodadi
392,marzuq,YE,14.4000,46.4667,53,No hotel found
456,avera,US,33.1940,-82.5271,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600 ,
    size= 'Humidity',
    color= 'City',
    hover_cols= ['Hotel Name' , 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)